# Kayseri Ev Fiyatları Tahmin Projesi

## Proje Açıklaması
Bu projede, Kayseri ilindeki ev fiyatlarını tahmin etmek amacıyla web scraping yöntemi kullanarak veriler topladık. Verileri çekmek için Python'un Beautiful Soup kütüphanesini kullandık. Elde edilen veriler üzerinde çeşitli makine öğrenimi modelleri uygulayarak, en iyi sonuçları elde etmeye çalıştık.

Kullanılan başlıca adımlar şunlardır:
1. **Veri Toplama**: Web scraping ile Kayseri'deki ev fiyatları ve özellikleri hakkında veriler toplandı.
2. **Veri Ön İşleme**: Elde edilen veriler temizlenerek, eksik değerler ve aykırı değerler kontrol edildi.
3. **Modelleme**: Farklı makine öğrenimi algoritmaları (özellikle Extra Tree Regressor) kullanılarak modeller oluşturuldu ve sonuçlar değerlendirildi.
4. **Sonuçların Değerlendirilmesi**: Elde edilen modellerin performansı R², RMSE ve MAE gibi metriklerle ölçüldü.

In [154]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("kayseri_kiralik_ev_data.csv")

In [3]:
df.head()

,İlan Numarası,İlan Oluşturma Tarihi,İlan Güncelleme Tarihi,Türü,Kategorisi,Tipi,Net Metrekare,Brüt Metrekare,Oda Sayısı,Binanın Yaşı,Bulunduğu Kat,Binanın Kat Sayısı,Isıtma Tipi,Kullanım Durumu,Eşya Durumu,Tapu Durumu,Site İçerisinde,Takas,Banyo Sayısı,Fiyat Durumu,Fiyat,Lokasyon,İlan Linki,Yapı Durumu,Yapı Tipi,Aidat,Depozito,Banyo Metrekare,Balkon Durumu,Balkon Sayısı,Balkon Tipi,Balkon Metrekare,Salon Metrekare,WC Sayısı,WC Metrekare,Görüntülü Gezilebilir mi?,Ada,Parsel
0,17087589,28 Mart 2025,28 Mart 2025,Konut,Kiralık,Daire,125 m²,165 m²,3+1,3,5.Kat,13,Merkezi (Pay Ölçer),Boş,Boş,Kat Mülkiyeti,Hayır,Yok,1,Genel Fiyat,17.000TL,Kayseri - Kocasinan,https://www.emlakjet.com/ilan/kw-ata-team-kira...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17082905,27 Mart 2025,27 Mart 2025,Konut,Kiralık,Daire,230 m²,260 m²,4+1,16-20,13.Kat,14,Merkezi Doğalgaz,Boş,Eşyalı,Kat Mülkiyeti,Hayır,Yok,2,Genel Fiyat,25.000TL,Kayseri - Melikgazi,https://www.emlakjet.com/ilan/alpaslan-mahalle...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17081763,27 Mart 2025,27 Mart 2025,Konut,Kiralık,Daire,125 m²,180 m²,3+1,16-20,9.Kat,10,Merkezi Doğalgaz,Boş,Boş,Kat İrtifakı,Hayır,Yok,2,Genel Fiyat,19.000TL,Kayseri - Melikgazi,https://www.emlakjet.com/ilan/aydinlikevler-ma...,İkinci El,Betonarme,650 TL,38000 TL,7 m²,Var,3.0,"Açık Balkon, Kapalı Balkon",5 m²,25 m²,2,5 m²,Hayır,736.0,27.0
3,17078664,26 Mart 2025,26 Mart 2025,Konut,Kiralık,Daire,75 m²,100 m²,2+1,2,6.Kat,10,Merkezi (Pay Ölçer),Boş,Boş,Kat Mülkiyeti,Hayır,Yok,1,Genel Fiyat,11.000TL,Kayseri - Kocasinan,https://www.emlakjet.com/ilan/sehir-hastanesi-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17076773,26 Mart 2025,26 Mart 2025,Konut,Kiralık,Daire,125 m²,170 m²,3+1,3,5.Kat,5,Kombi Doğalgaz,Boş,Boş,Kat Mülkiyeti,Hayır,Yok,2,Genel Fiyat,13.000TL,Kayseri - Melikgazi,https://www.emlakjet.com/ilan/melikgazi-bahcel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.isnull().sum()

İlan Numarası                  0
İlan Oluşturma Tarihi          0
İlan Güncelleme Tarihi         0
Türü                           0
Kategorisi                     0
Tipi                           0
Net Metrekare                  1
Brüt Metrekare                 0
Oda Sayısı                     0
Binanın Yaşı                   0
Bulunduğu Kat                 24
Binanın Kat Sayısı             0
Isıtma Tipi                    0
Kullanım Durumu                0
Eşya Durumu                  285
Tapu Durumu                    0
Site İçerisinde                0
Takas                        203
Banyo Sayısı                   0
Fiyat Durumu                   0
Fiyat                          0
Lokasyon                       0
İlan Linki                     0
Yapı Durumu                  840
Yapı Tipi                    874
Aidat                        876
Depozito                     897
Banyo Metrekare              959
Balkon Durumu                908
Balkon Sayısı                901
Balkon Tip

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   İlan Numarası              969 non-null    int64  
 1   İlan Oluşturma Tarihi      969 non-null    object 
 2   İlan Güncelleme Tarihi     969 non-null    object 
 3   Türü                       969 non-null    object 
 4   Kategorisi                 969 non-null    object 
 5   Tipi                       969 non-null    object 
 6   Net Metrekare              968 non-null    object 
 7   Brüt Metrekare             969 non-null    object 
 8   Oda Sayısı                 969 non-null    object 
 9   Binanın Yaşı               969 non-null    object 
 10  Bulunduğu Kat              945 non-null    object 
 11  Binanın Kat Sayısı         969 non-null    int64  
 12  Isıtma Tipi                969 non-null    object 
 13  Kullanım Durumu            969 non-null    object 

In [6]:
null_percentage = (df.isnull().sum() / len(df)) * 100
print(null_percentage)

İlan Numarası                 0.000000
İlan Oluşturma Tarihi         0.000000
İlan Güncelleme Tarihi        0.000000
Türü                          0.000000
Kategorisi                    0.000000
Tipi                          0.000000
Net Metrekare                 0.103199
Brüt Metrekare                0.000000
Oda Sayısı                    0.000000
Binanın Yaşı                  0.000000
Bulunduğu Kat                 2.476780
Binanın Kat Sayısı            0.000000
Isıtma Tipi                   0.000000
Kullanım Durumu               0.000000
Eşya Durumu                  29.411765
Tapu Durumu                   0.000000
Site İçerisinde               0.000000
Takas                        20.949432
Banyo Sayısı                  0.000000
Fiyat Durumu                  0.000000
Fiyat                         0.000000
Lokasyon                      0.000000
İlan Linki                    0.000000
Yapı Durumu                  86.687307
Yapı Tipi                    90.196078
Aidat                    

In [7]:
null_percentage = (df.isnull().sum() / len(df)) * 100
threshold = 85
df = df.drop(columns=null_percentage[null_percentage > threshold].index)

In [8]:
df.isnull().sum()

İlan Numarası               0
İlan Oluşturma Tarihi       0
İlan Güncelleme Tarihi      0
Türü                        0
Kategorisi                  0
Tipi                        0
Net Metrekare               1
Brüt Metrekare              0
Oda Sayısı                  0
Binanın Yaşı                0
Bulunduğu Kat              24
Binanın Kat Sayısı          0
Isıtma Tipi                 0
Kullanım Durumu             0
Eşya Durumu               285
Tapu Durumu                 0
Site İçerisinde             0
Takas                     203
Banyo Sayısı                0
Fiyat Durumu                0
Fiyat                       0
Lokasyon                    0
İlan Linki                  0
dtype: int64

In [9]:
df['Türü'].unique()

array(['Konut'], dtype=object)

In [10]:
df['Türü'].nunique() == 1
# Tüm değerler aynıysa True, değilse False döner

True

In [11]:
df = df.drop(columns=['Türü'])

In [12]:
df['Kategorisi'].unique()

array(['Kiralık'], dtype=object)

In [13]:
df['Kategorisi'].nunique() == 1

True

In [14]:
df = df.drop(columns=['Kategorisi'])

In [15]:
df['Tipi'].unique()

array(['Daire', 'Müstakil Ev', 'Villa', 'Köşk', 'Residence', 'Bina'],
      dtype=object)

In [16]:
df['Net Metrekare'].unique()

array(['125 m²', '230 m²', '75 m²', '100 m²', '115 m²', '140 m²', '85 m²',
       '130 m²', '185 m²', '135 m²', '165 m²', '25 m²', '162 m²',
       '120 m²', '90 m²', '110 m²', '160 m²', '220 m²', '150 m²',
       '145 m²', '175 m²', '200 m²', '148 m²', '60 m²', '155 m²', '55 m²',
       '70 m²', '95 m²', '80 m²', '170 m²', '190 m²', '143 m²', '225 m²',
       '105 m²', '180 m²', '370 m²', '76 m²', '116 m²', '12 m²', '128 m²',
       '163 m²', '50 m²', '132 m²', '122 m²', '112 m²', '65 m²', '159 m²',
       '550 m²', '68 m²', '30 m²', '700 m²', '300 m²', '127 m²', '87 m²',
       '118 m²', '138 m²', '198 m²', '45 m²', '117 m²', '124 m²',
       '187 m²', '260 m²', '133 m²', nan, '149 m²', '144 m²', '210 m²'],
      dtype=object)

In [17]:
df['Net Metrekare'] = df['Net Metrekare'].str.replace(' m²', '', regex=False).astype(float)

In [18]:
df['Net Metrekare'].unique()

array([125., 230.,  75., 100., 115., 140.,  85., 130., 185., 135., 165.,
        25., 162., 120.,  90., 110., 160., 220., 150., 145., 175., 200.,
       148.,  60., 155.,  55.,  70.,  95.,  80., 170., 190., 143., 225.,
       105., 180., 370.,  76., 116.,  12., 128., 163.,  50., 132., 122.,
       112.,  65., 159., 550.,  68.,  30., 700., 300., 127.,  87., 118.,
       138., 198.,  45., 117., 124., 187., 260., 133.,  nan, 149., 144.,
       210.])

In [19]:
df['Net Metrekare'].describe()

count    968.000000
mean     123.913223
std       40.681865
min       12.000000
25%      100.000000
50%      125.000000
75%      140.000000
max      700.000000
Name: Net Metrekare, dtype: float64

In [20]:
df['Net Metrekare'].mode()

0    135.0
Name: Net Metrekare, dtype: float64

In [21]:
df['Net Metrekare'].fillna(df['Net Metrekare'].mode()[0], inplace=True)

In [22]:
df['Brüt Metrekare'].unique()

array(['165 m²', '260 m²', '180 m²', '100 m²', '170 m²', '175 m²',
       '135 m²', '140 m²', '185 m²', '130 m²', '145 m²', '120 m²',
       '210 m²', '155 m²', '195 m²', '150 m²', '110 m²', '30 m²',
       '115 m²', '103 m²', '105 m²', '220 m²', '160 m²', '200 m²',
       '225 m²', '80 m²', '85 m²', '240 m²', '169 m²', '90 m²', '205 m²',
       '65 m²', '125 m²', '128 m²', '250 m²', '92 m²', '190 m²', '75 m²',
       '280 m²', '142 m²', '600 m²', '119 m²', '95 m²', '14 m²', '132 m²',
       '215 m²', '40 m²', '153 m²', '750 m²', '70 m²', '450 m²', '149 m²',
       '50 m²', '136 m²', '360 m²', '235 m²', '245 m²', '350 m²',
       '530 m²', '82 m²', '270 m²', '370 m²', '147 m²'], dtype=object)

In [23]:
df['Brüt Metrekare'] = df['Brüt Metrekare'].str.replace(' m²', '', regex=False).astype(float)

In [24]:
df['Oda Sayısı'].unique()

array(['3+1', '4+1', '2+1', '2.5+1', 'Stüdyo', '1+1', '4.5+1', '5+1',
       '3.5+1', '4+0', '6+1', '1 Oda', '7+2', '3+0', '6+2', '4+2',
       '9+ Oda', '8+1', '3+2', '8+3', '7+1'], dtype=object)

In [25]:
df = df[~df['Oda Sayısı'].str.contains('Stüdyo', na=False)]

In [26]:
import re

# Oda sayısını çıkarma (+, öncesi)
df['Oda'] = df['Oda Sayısı'].str.split('+').str[0].str.strip()

# + içermeyenler için (ör. '1 Oda') düzeltme
mask = ~df['Oda Sayısı'].str.contains('+', regex=False)
df.loc[mask, 'Oda'] = df.loc[mask, 'Oda Sayısı'].str.split().str[0]
df['Oda'] = df['Oda'].astype(float)

# Salon sayısını çıkarma (+, sonrası)
def extract_salon(oda_str):
    if '+' in oda_str:
        right_part = oda_str.split('+')[1].strip()
        return 1 if 'Oda' in right_part else int(re.search(r'\d+', right_part).group(0) if re.search(r'\d', right_part) else 0)
    return 0

df['Salon'] = df['Oda Sayısı'].apply(extract_salon).astype(int)

In [27]:
df['Oda'].unique()

array([3. , 4. , 2. , 2.5, 1. , 4.5, 5. , 3.5, 6. , 7. , 9. , 8. ])

In [28]:
df['Salon'].unique()

array([1, 0, 2, 3])

In [29]:
df['Binanın Yaşı'].unique()

array(['3', '16-20', '2', '4', '0 (Yeni)', '21 Ve Üzeri', '5-10', '11-15',
       '1'], dtype=object)

In [30]:
df['Binanın Yaşı'] = df['Binanın Yaşı'].replace('0 (Yeni)', 0)
df['Binanın Yaşı'] = df['Binanın Yaşı'].replace('21 Ve Üzeri', 30)
df['Binanın Yaşı'] = df['Binanın Yaşı'].replace('16-20', '18')
df['Binanın Yaşı'] = df['Binanın Yaşı'].replace('11-15', '13')
df['Binanın Yaşı'] = df['Binanın Yaşı'].replace('5-10', '7')
df['Binanın Yaşı'] = df['Binanın Yaşı'].astype(int)

In [31]:
df['Binanın Yaşı'].unique()

array([ 3, 18,  2,  4,  0, 30,  7, 13,  1])

In [32]:
df['Bulunduğu Kat'].unique()

array(['5.Kat', '13.Kat', '9.Kat', '6.Kat', '4.Kat', '2.Kat', '3.Kat',
       'Yüksek Giriş', '8.Kat', 'Düz Giriş (Zemin)', '10.Kat',
       'Çatı Katı', 'Kot 4 (-4).Kat', '11.Kat', '1.Kat', '7.Kat',
       'Müstakil', '12.Kat', '14.Kat', 'Kot 1 (-1).Kat', 'Villa Tipi',
       '15.Kat', nan, 'Bodrum Kat', 'Kot 3 (-3).Kat', 'Kot 2 (-2).Kat',
       '20.Kat'], dtype=object)

In [33]:
kat_mapping = {
    'Müstakil': 1,
    'Villa Tipi': 1,
    'Yüksek Giriş': 1,
    'Düz Giriş (Zemin)': 0,
    'Bodrum Kat': -1,
    'Çatı Katı': 'cati_placeholder',  # Geçici string işaret
    'Zemin': 0
}

In [34]:
df['Kat'] = (
    df['Bulunduğu Kat']
    .map(kat_mapping)
    .fillna(df['Bulunduğu Kat']) 
)

kot_mask = df['Bulunduğu Kat'].str.contains('Kot', na=False)
df.loc[kot_mask, 'Kat'] = (
    df.loc[kot_mask, 'Bulunduğu Kat']
    .str.extract(r'\((-?\d+)\)')[0]
    .astype(float)
)

In [35]:
normal_kat_mask = (
    df['Bulunduğu Kat'].str.match(r'^\d+\.?Kat', na=False) &
    ~kot_mask  
)
df.loc[normal_kat_mask, 'Kat'] = (
    df.loc[normal_kat_mask, 'Bulunduğu Kat']
    .str.extract(r'^(\d+)\.?Kat')[0]
    .astype(float)
)

In [36]:
df['Kat'] = (
    df['Kat']
    .replace('cati_placeholder', np.nan) 
    .apply(pd.to_numeric, errors='coerce')  
)

In [37]:
df['Kat'].max(skipna=True)

20.0

In [38]:
max_kat = 20
df['Kat'] = df['Kat'].fillna(max_kat)

In [39]:
df['Kat'] = df['Kat'].mask(df['Kat'] < 0, 0).astype(int)

In [40]:
mode_kat = df['Kat'].mode()[0]
df['Kat'] = df['Kat'].fillna(mode_kat)

In [41]:
df['Kat'].unique()

array([ 5, 13,  9,  6,  4,  2,  3,  1,  8,  0, 10, 20, 11,  7, 12, 14, 15])

In [42]:
df['Binanın Kat Sayısı'].unique()

array([13, 14, 10,  5, 11,  6,  2, 12,  3,  9, 15,  4,  8,  7,  1, 18, 21,
       19], dtype=int64)

In [43]:
df['Isıtma Tipi'].unique()

array(['Merkezi (Pay Ölçer)', 'Merkezi Doğalgaz', 'Kombi Doğalgaz',
       'Sobalı', 'Yerden Isıtma', 'Kat Kaloriferi', 'Isıtma Yok',
       'Doğalgaz Sobalı', 'Merkezi Fueloil'], dtype=object)

In [44]:
isitma_mapping = {
    # Merkezi Grup
    'Merkezi (Pay Ölçer)': 'Merkezi',
    'Merkezi Doğalgaz': 'Merkezi',
    'Merkezi Fueloil': 'Merkezi',
    
    # Doğalgaz Grup
    'Kombi Doğalgaz': 'Doğalgaz',
    'Kat Kaloriferi': 'Doğalgaz',
    'Doğalgaz Sobalı': 'Doğalgaz',
    
    # Sobalı Grup
    'Sobalı': 'Sobalı',
    'Isıtma Yok': 'Sobalı'
}

df['Isıtma_Kategori'] = df['Isıtma Tipi'].replace(isitma_mapping)

df.loc[df['Isıtma Tipi'] == 'Yerden Isıtma', 'Isıtma_Kategori'] = 'Yerden Isıtma'

In [45]:
df['Isıtma_Kategori'].unique()

array(['Merkezi', 'Doğalgaz', 'Sobalı', 'Yerden Isıtma'], dtype=object)

In [46]:
df['Kullanım Durumu'].unique()

array(['Boş', 'Kiracı Oturuyor', 'Mülk Sahibi Oturuyor'], dtype=object)

In [47]:
df['Eşya Durumu'].unique()

array(['Boş', 'Eşyalı', nan], dtype=object)

In [48]:
df['Eşya Durumu'].fillna('Boş', inplace=True)

In [49]:
df['Eşya Durumu'] = df['Eşya Durumu'].map({
    'Boş': 0,
    'Eşyalı': 1
})

In [50]:
df['Tapu Durumu'].unique()

array(['Kat Mülkiyeti', 'Kat İrtifakı', 'Tapu Kaydı Yok', 'Hisseli Tapu',
       'Kooperatiften Tapu', 'Müstakil Tapulu', 'Bilinmiyor',
       'Arsa Tapulu'], dtype=object)

In [51]:
df['Tapu Durumu'] = df['Tapu Durumu'].map({
    'Tapu Kaydı Yok': 0,
    'Kat Mülkiyeti': 1,
    'Kat İrtifakı': 1,
    'Hisseli Tapu': 1,
    'Kooperatiften Tapu': 1,
    'Müstakil Tapulu': 1,
    'Bilinmiyor': 1,
    'Arsa Tapulu': 1
})

In [52]:
df['Tapu Durumu'].unique()

array([1, 0], dtype=int64)

In [53]:
df['Site İçerisinde'].unique()

array(['Hayır', 'Evet'], dtype=object)

In [54]:
df['Site İçerisinde'] = df['Site İçerisinde'].map({
    'Hayır': 0,
    'Evet': 1
})

In [55]:
df['Banyo Sayısı'].unique()

array(['1', '2', '3', '4', 'Yok', '5'], dtype=object)

In [56]:
df['Banyo Sayısı'] = df['Banyo Sayısı'].map({
    'Yok': 0
}).fillna(df['Banyo Sayısı'])

In [57]:
df['Banyo Sayısı'] = df['Banyo Sayısı'].astype(float)

In [58]:
df['Takas'].unique()

array(['Yok', nan, 'Var'], dtype=object)

In [59]:
df['Takas'].fillna('Yok', inplace=True)

In [60]:
df['Fiyat Durumu'].unique()

array(['Genel Fiyat', 'Birim Fiyat'], dtype=object)

In [61]:
df['Fiyat'].unique()

array(['17.000TL', '25.000TL', '19.000TL', '11.000TL', '13.000TL',
       '20.000TL', '10.500TL', '21.000TL', '15.000TL', '11.500TL',
       '22.000TL', '6.500TL', '10.700TL', '8.000TL', '25.500TL',
       '10.000TL', '13.500TL', '18.000TL', '6.000TL', '9.000TL',
       '15.500TL', '18.500TL', '12.000TL', '14.250TL', '7.000TL',
       '8.500TL', '35.000TL', '14.000TL', '9.500TL', '6.999TL',
       '12.500TL', '23.000TL', '23.500TL', '30.000TL', '26.000TL',
       '24.000TL', '16.500TL', '16.000TL', '19.999TL', '13.400TL',
       '7.500TL', '9.250TL', '12.850TL', '27.500TL', '70.000TL',
       '12.950TL', '19.500TL', '10.250TL', '27.000TL', '22.500TL',
       '14.500TL', '12.750TL', '16.750TL', '12.250TL', '8.750TL',
       '17.500TL', '7.750TL', '50.000TL', '5.000TL', '42.500TL',
       '90.000TL', '40.000TL', '12.400TL', '13.300TL', '5.500TL',
       '9.750TL', '28.000TL', '11.250TL', '13.750TL', '75.000TL',
       '15.250TL', '60.000TL', '29.000TL', '7.250TL', '12.900TL',
       '58.

In [62]:
df['Fiyat'] = (
    df['Fiyat']
    .str.replace('TL', '', regex=False)    
    .str.replace('.', '', regex=False)       
    .str.replace(',', '.', regex=False) 
    .str.strip()                         
)

In [63]:
df['Fiyat'] = pd.to_numeric(df['Fiyat'], errors='coerce')

In [64]:
df['Fiyat'].unique()

array([  17000,   25000,   19000,   11000,   13000,   20000,   10500,
         21000,   15000,   11500,   22000,    6500,   10700,    8000,
         25500,   10000,   13500,   18000,    6000,    9000,   15500,
         18500,   12000,   14250,    7000,    8500,   35000,   14000,
          9500,    6999,   12500,   23000,   23500,   30000,   26000,
         24000,   16500,   16000,   19999,   13400,    7500,    9250,
         12850,   27500,   70000,   12950,   19500,   10250,   27000,
         22500,   14500,   12750,   16750,   12250,    8750,   17500,
          7750,   50000,    5000,   42500,   90000,   40000,   12400,
         13300,    5500,    9750,   28000,   11250,   13750,   75000,
         15250,   60000,   29000,    7250,   12900,   58000, 1400000,
         10750,   12200,    6800,    1200,   45000,   15750,   33000,
         29500, 7500000,   14750,   13900,   95000,    8900,  100000,
         34500,   11999,  120000,   46000,   65000,  250000,   13730],
      dtype=int64)

In [65]:
df['Lokasyon'].unique()

array(['Kayseri - Kocasinan ', 'Kayseri - Melikgazi ', 'Kayseri - Talas ',
       'Kayseri - Develi ', 'Kayseri - İncesu ', 'Kayseri - Yeşilhisar '],
      dtype=object)

In [66]:
df['Lokasyon'] = df['Lokasyon'].str.split(' - ').str[1].str.strip()

In [67]:
df['Lokasyon'].unique()

array(['Kocasinan', 'Melikgazi', 'Talas', 'Develi', 'İncesu',
       'Yeşilhisar'], dtype=object)

In [68]:
df = df.drop(columns=['İlan Oluşturma Tarihi', 
                      'İlan Güncelleme Tarihi', 
                      'Kullanım Durumu', 
                      'İlan Linki', 
                      'Fiyat Durumu', 
                      'Oda Sayısı',
                      'İlan Numarası',
                      'Bulunduğu Kat'])

In [69]:
df.isnull().sum()

Tipi                  0
Net Metrekare         0
Brüt Metrekare        0
Binanın Yaşı          0
Binanın Kat Sayısı    0
Isıtma Tipi           0
Eşya Durumu           0
Tapu Durumu           0
Site İçerisinde       0
Takas                 0
Banyo Sayısı          0
Fiyat                 0
Lokasyon              0
Oda                   0
Salon                 0
Kat                   0
Isıtma_Kategori       0
dtype: int64

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 965 entries, 0 to 968
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Tipi                965 non-null    object 
 1   Net Metrekare       965 non-null    float64
 2   Brüt Metrekare      965 non-null    float64
 3   Binanın Yaşı        965 non-null    int32  
 4   Binanın Kat Sayısı  965 non-null    int64  
 5   Isıtma Tipi         965 non-null    object 
 6   Eşya Durumu         965 non-null    int64  
 7   Tapu Durumu         965 non-null    int64  
 8   Site İçerisinde     965 non-null    int64  
 9   Takas               965 non-null    object 
 10  Banyo Sayısı        965 non-null    float64
 11  Fiyat               965 non-null    int64  
 12  Lokasyon            965 non-null    object 
 13  Oda                 965 non-null    float64
 14  Salon               965 non-null    int32  
 15  Kat                 965 non-null    int32  
 16  Isıtma_Katego

In [71]:
df.corr(numeric_only=True)

,Net Metrekare,Brüt Metrekare,Binanın Yaşı,Binanın Kat Sayısı,Eşya Durumu,Tapu Durumu,Site İçerisinde,Banyo Sayısı,Fiyat,Oda,Salon,Kat
Net Metrekare,1.000000,0.871674,0.075400,0.109984,-0.094365,0.030107,0.126336,0.501233,0.043827,0.768159,0.183077,0.298899
Brüt Metrekare,0.871674,1.000000,0.057206,0.090414,-0.082277,0.024392,0.123858,0.545519,0.050416,0.772286,0.344889,0.352597
Binanın Yaşı,0.075400,0.057206,1.000000,-0.315824,0.005837,-0.006189,-0.032841,-0.281576,-0.029504,0.024153,0.087504,-0.138563
Binanın Kat Sayısı,0.109984,0.090414,-0.315824,1.000000,-0.014201,0.010019,0.148955,0.210159,-0.057582,0.114833,-0.080351,0.278355
Eşya Durumu,-0.094365,-0.082277,0.005837,-0.014201,1.000000,0.047341,0.010433,-0.107766,-0.007038,-0.189147,0.031068,0.026426
Tapu Durumu,0.030107,0.024392,-0.006189,0.010019,0.047341,1.000000,-0.005861,0.026673,0.009021,-0.012177,-0.002348,0.032939
Site İçerisinde,0.126336,0.123858,-0.032841,0.148955,0.010433,-0.005861,1.000000,0.123688,-0.001438,0.092480,0.005031,0.002990
Banyo Sayısı,0.501233,0.545519,-0.281576,0.210159,-0.107766,0.026673,0.123688,1.000000,0.047633,0.536095,0.172290,0.285901
Fiyat,0.043827,0.050416,-0.029504,-0.057582,-0.007038,0.009021,-0.001438,0.047633,1.000000,0.084923,0.011640,0.102350
Oda,0.768159,0.772286,0.024153,0.114833,-0.189147,-0.012177,0.092480,0.536095,0.084923,1.000000,0.187139,0.288852


In [72]:
df = pd.get_dummies(df, drop_first=True)

In [73]:
df.head()

,Net Metrekare,Brüt Metrekare,Binanın Yaşı,Binanın Kat Sayısı,Eşya Durumu,Tapu Durumu,Site İçerisinde,Banyo Sayısı,Fiyat,Oda,Salon,Kat,Tipi_Daire,Tipi_Köşk,Tipi_Müstakil Ev,Tipi_Residence,Tipi_Villa,Isıtma Tipi_Isıtma Yok,Isıtma Tipi_Kat Kaloriferi,Isıtma Tipi_Kombi Doğalgaz,Isıtma Tipi_Merkezi (Pay Ölçer),Isıtma Tipi_Merkezi Doğalgaz,Isıtma Tipi_Merkezi Fueloil,Isıtma Tipi_Sobalı,Isıtma Tipi_Yerden Isıtma,Takas_Yok,Lokasyon_Kocasinan,Lokasyon_Melikgazi,Lokasyon_Talas,Lokasyon_Yeşilhisar,Lokasyon_İncesu,Isıtma_Kategori_Merkezi,Isıtma_Kategori_Sobalı,Isıtma_Kategori_Yerden Isıtma
0,125.0,165.0,3,13,0,1,0,1.0,17000,3.0,1,5,True,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False
1,230.0,260.0,18,14,1,1,0,2.0,25000,4.0,1,13,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,True,False,False
2,125.0,180.0,18,10,0,1,0,2.0,19000,3.0,1,9,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,True,False,False
3,75.0,100.0,2,10,0,1,0,1.0,11000,2.0,1,6,True,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False
4,125.0,170.0,3,5,0,1,0,2.0,13000,3.0,1,5,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False


In [74]:
df.corr()

,Net Metrekare,Brüt Metrekare,Binanın Yaşı,Binanın Kat Sayısı,Eşya Durumu,Tapu Durumu,Site İçerisinde,Banyo Sayısı,Fiyat,Oda,Salon,Kat,Tipi_Daire,Tipi_Köşk,Tipi_Müstakil Ev,Tipi_Residence,Tipi_Villa,Isıtma Tipi_Isıtma Yok,Isıtma Tipi_Kat Kaloriferi,Isıtma Tipi_Kombi Doğalgaz,Isıtma Tipi_Merkezi (Pay Ölçer),Isıtma Tipi_Merkezi Doğalgaz,Isıtma Tipi_Merkezi Fueloil,Isıtma Tipi_Sobalı,Isıtma Tipi_Yerden Isıtma,Takas_Yok,Lokasyon_Kocasinan,Lokasyon_Melikgazi,Lokasyon_Talas,Lokasyon_Yeşilhisar,Lokasyon_İncesu,Isıtma_Kategori_Merkezi,Isıtma_Kategori_Sobalı,Isıtma_Kategori_Yerden Isıtma
Net Metrekare,1.000000,0.871674,0.075400,0.109984,-0.094365,0.030107,0.126336,0.501233,0.043827,0.768159,0.183077,0.298899,-0.296792,0.458562,0.075339,-0.112831,0.337583,-0.035223,0.019481,-0.012438,-0.048998,0.050060,0.004599,0.005483,0.095429,0.003366,0.007484,0.123350,-0.083286,-0.046625,-0.063372,-0.000219,-0.002414,0.095429
Brüt Metrekare,0.871674,1.000000,0.057206,0.090414,-0.082277,0.024392,0.123858,0.545519,0.050416,0.772286,0.344889,0.352597,-0.400305,0.357493,0.101477,-0.098915,0.498979,-0.030995,0.011889,-0.013030,-0.028896,0.027752,-0.010077,0.020355,0.087883,0.010077,0.023468,0.125677,-0.079010,-0.050116,-0.058923,-0.002738,0.013040,0.087883
Binanın Yaşı,0.075400,0.057206,1.000000,-0.315824,0.005837,-0.006189,-0.032841,-0.281576,-0.029504,0.024153,0.087504,-0.138563,-0.031242,0.064470,0.101618,-0.017465,-0.044271,0.064470,-0.020411,0.311879,-0.478362,0.103141,-0.025031,0.273233,-0.054896,-0.008117,0.067092,0.080736,-0.146211,0.129140,-0.031906,-0.385111,0.281006,-0.054896
Binanın Kat Sayısı,0.109984,0.090414,-0.315824,1.000000,-0.014201,0.010019,0.148955,0.210159,-0.057582,0.114833,-0.080351,0.278355,0.244212,-0.054591,-0.173443,-0.013919,-0.168257,-0.031913,0.066995,-0.605675,0.379756,0.280320,0.043679,-0.224386,-0.042226,-0.021001,-0.117486,0.018011,0.345388,-0.105566,-0.085915,0.662615,-0.226132,-0.042226
Eşya Durumu,-0.094365,-0.082277,0.005837,-0.014201,1.000000,0.047341,0.010433,-0.107766,-0.007038,-0.189147,0.031068,0.026426,-0.016915,-0.009350,0.046736,0.032961,-0.031172,-0.009350,-0.016211,0.009673,-0.044555,0.047531,-0.009350,-0.015063,-0.016211,0.009350,-0.054619,-0.071178,0.156244,-0.018729,-0.016211,0.000850,-0.016769,-0.016211
Tapu Durumu,0.030107,0.024392,-0.006189,0.010019,0.047341,1.000000,-0.005861,0.026673,0.009021,-0.012177,-0.002348,0.032939,-0.028706,0.005253,0.016688,0.011769,0.017512,-0.197495,0.009107,-0.019607,-0.015909,0.039068,0.005253,0.023725,0.009107,-0.005253,0.040054,-0.136956,0.078458,0.010521,0.009107,0.022562,-0.020386,0.009107
Site İçerisinde,0.126336,0.123858,-0.032841,0.148955,0.010433,-0.005861,1.000000,0.123688,-0.001438,0.092480,0.005031,0.002990,0.042013,-0.011254,-0.035756,-0.025217,-0.006172,-0.011254,-0.019513,-0.086687,0.054972,0.045709,-0.011254,-0.050833,0.040264,-0.092176,0.000223,-0.003742,0.061635,-0.022543,0.040264,0.099752,-0.052116,0.040264
Banyo Sayısı,0.501233,0.545519,-0.281576,0.210159,-0.107766,0.026673,0.123688,1.000000,0.047633,0.536095,0.172290,0.285901,-0.199364,0.088375,-0.002569,-0.051771,0.328426,-0.023105,-0.007845,-0.135851,0.178781,-0.032362,-0.023105,-0.079183,0.088799,0.023105,0.005426,0.097271,-0.099992,-0.046281,-0.040060,0.147784,-0.082411,0.088799
Fiyat,0.043827,0.050416,-0.029504,-0.057582,-0.007038,0.009021,-0.001438,0.047633,1.000000,0.084923,0.011640,0.102350,-0.192560,0.004641,-0.002466,-0.004349,0.314229,-0.001925,-0.002806,-0.026633,-0.024541,0.042111,-0.001400,0.031579,0.006718,0.001433,-0.014249,0.037798,-0.016017,-0.004711,-0.004002,0.016349,0.030409,0.006718
Oda,0.768159,0.772286,0.024153,0.114833,-0.189147,-0.012177,0.092480,0.536095,0.084923,1.000000,0.187139,0.288852,-0.261374,0.257730,0.053305,-0.123431,0.372398,-0.038177,0.007102,-0.028114,0.008649,0.004606,0.004096,-0.010141,0.068180,-0.004096,0.065521,0.107633,-0.138143,-0.055303,-0.066193,0.013558,-0.018320,0.068180


In [75]:
def algo_test(x, y):
    # Modellerin tanımlanması
    models = [
        ('Linear Regression', LinearRegression()),
        ('Ridge', Ridge()),
        ('Lasso', Lasso()),
        ('ElasticNet', ElasticNet()),
        ('SGD Regressor', SGDRegressor()),
        ('Extra Tree', ExtraTreeRegressor()),
        ('Gradient Boosting', GradientBoostingRegressor()),
        ('AdaBoost', AdaBoostRegressor()),
        ('KNN', KNeighborsRegressor()),
        ('Decision Tree', DecisionTreeRegressor()),
        ('XGBoost', XGBRegressor()),
        ('SVR', SVR()),
        ('MLP Regressor', MLPRegressor(max_iter=1000))
    ]
    
    # Verinin bölünmesi
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    # Ölçeklendirme
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Sonuç DataFrame'i
    results = pd.DataFrame(columns=['Model', 'R2', 'RMSE', 'MAE'])
    
    for name, model in models:
        try:
            # Model eğitimi ve tahmin
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            
            # Metrik hesaplamaları
            r2 = round(r2_score(y_test, y_pred), 4)
            rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 2)
            mae = round(mean_absolute_error(y_test, y_pred), 2)
            
            # Sonuçların kaydedilmesi
            results = pd.concat([results, pd.DataFrame({
                'Model': [name],
                'R2': [r2],
                'RMSE': [rmse],
                'MAE': [mae]
            })], ignore_index=True)
            
        except Exception as e:
            print(f"{name} modelinde hata oluştu: {str(e)}")
    
    return results.sort_values('R2', ascending=False)

# Kullanım örneği
X = df.drop('Fiyat', axis=1)
y = df['Fiyat']

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
X_processed = pd.get_dummies(X, columns=categorical_cols)

# Algoritma testinin çalıştırılması
performance_results = algo_test(X_processed, y)

# Sonuçların gösterilmesi
print("="*60)
print("MODEL PERFORMANS KARŞILAŞTIRMASI")
print("="*60)
print(performance_results.to_string(index=False))

MODEL PERFORMANS KARŞILAŞTIRMASI
            Model         R2      RMSE      MAE
       Extra Tree     0.7489   4223.80  2773.58
         AdaBoost     0.2019   7529.73  3734.92
              SVR    -0.0307   8556.57  4116.72
    MLP Regressor    -1.3890  13027.24 10718.42
       ElasticNet   -99.8273  84631.08 30806.32
    SGD Regressor  -129.0766  96125.90 42410.33
    Decision Tree  -148.9947 103223.54 12732.90
Gradient Boosting  -243.8439 131881.96 15623.71
            Ridge  -257.6084 135538.31 51374.49
            Lasso  -259.4995 136032.99 51548.62
Linear Regression  -259.5208 136038.54 51551.42
              KNN  -507.3589 190031.68 28520.47
          XGBoost -4045.6913 536155.57 42944.25


## Sonuçlar
En iyi sonuçları elde ettiğimiz model, **Extra Tree Regressor** oldu:
- **R²**: 0.7489
- **RMSE**: 4223.80
- **MAE**: 2773.58

Bu sonuçlar, modelimizin Kayseri ev fiyatlarını tahmin etmede daha fazla iyileştirme yapılması gerektiğini ortaya koymaktadır.